##### 1.预处理 requirements/configure/tokenizer/dataloader

###### 1.1 Requirements & Imports

In [6]:
import math
import time
import spacy
import os
import torch
import numpy as np
import random
# HuggingFace tokenizers 核心类
from datasets import load_dataset
from tokenizers import Tokenizer as HFTokenizer
## BPE 模型（Byte Pair Encoding）
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers import pre_tokenizers, decoders

from collections import Counter, OrderedDict
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [7]:
# 固定随机种子，保证结果可复现
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(1234)

###### 1.2 Transformer 配置参数

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 模型参数
batch_size = 128 # 训练批次 句话
max_len = 256    # 单句最大长度
##
# padding=10

d_model = 512    # 词嵌入向量维度
n_layers = 6     # encoder/decoder层数量
n_heads = 8      # 注意力头数： 假如有词嵌入维度d_model = 512 / n_heads = 8 => 单头向量维度 512 / 8 = 64，即QKV维度
ffn_hidden = 2048 # 前向传播维度。 512 -> 2048 -> 512, 通常也称作proj
drop_prob = 0.1  # dropout提升鲁棒性，随机失活一些节点
n_hidden = ffn_hidden

# optimizer parameter setting
init_lr = 1e-5
factor = 0.9
adam_eps = 5e-9
patience = 10
warmup = 100
epoch = 100
clip = 1.0
weight_decay = 5e-4
inf = float('inf')


###### 1.3 Tokenizer 英德文 Tokenizer

In [9]:
import os
from datasets import load_dataset
from tokenizers import Tokenizer as HFTokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers import pre_tokenizers, decoders


class BBPETokenizerManager:
    def __init__(self, vocab_size=10000, min_freq=2):
        self.vocab_size = vocab_size
        self.min_freq = min_freq
        self.special_tokens = ["<pad>", "<sos>", "<eos>", "<unk>"]

        self.en_path = "tokenizer_en_bbpe.json"
        self.de_path = "tokenizer_de_bbpe.json"

        # 加载数据集 (替代 torchtext.datasets.Multi30k)
        # bentrevett/multi30k 是 Multi30k 的官方 HF 镜像版本
        print("正在从 Hugging Face Hub 加载 Multi30k 数据集...")
        self.dataset = load_dataset("bentrevett/multi30k")

        if os.path.exists(self.en_path) and os.path.exists(self.de_path):
            print(f"检测到已训练的 BBPE 分词器，正在加载...")
            self.tokenizer_en = HFTokenizer.from_file(self.en_path)
            self.tokenizer_de = HFTokenizer.from_file(self.de_path)
        else:
            print(f"开始训练 BBPE 分词器...")
            self.tokenizer_en, self.tokenizer_de = self.train_tokenizers()

    def train_tokenizers(self):
        # 1. 定义数据生成器 (适配 HF Dataset 格式: dict)
        def batch_iterator(dataset, lang):
            for i in range(0, len(dataset), 1000):
                yield dataset[i: i + 1000][lang]

        # 2. 训练逻辑
        def train(lang):
            tokenizer = HFTokenizer(BPE(unk_token="<unk>"))
            tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
            tokenizer.decoder = decoders.ByteLevel()

            trainer = BpeTrainer(
                vocab_size=self.vocab_size,
                min_frequency=self.min_freq,
                special_tokens=self.special_tokens,
                initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
            )

            print(f"正在训练 {lang} BBPE 分词器...")
            # 使用 train 数据集进行训练
            tokenizer.train_from_iterator(batch_iterator(self.dataset['train'], lang), trainer=trainer)
            return tokenizer

        tokenizer_de = train('de')
        tokenizer_en = train('en')

        tokenizer_de.save(self.de_path)
        tokenizer_en.save(self.en_path)
        print("BBPE 分词器训练完成。")

        return tokenizer_en, tokenizer_de

    def tokenize_en(self, text):
        return self.tokenizer_en.encode(text).tokens

    def tokenize_de(self, text):
        return self.tokenizer_de.encode(text).tokens

    # [New] 获取原始数据集的接口
    def get_dataset(self):
        return self.dataset


# 实例化
tokenizer_manager = BBPETokenizerManager(vocab_size=10000, min_freq=2)
raw_dataset = tokenizer_manager.get_dataset()


正在从 Hugging Face Hub 加载 Multi30k 数据集...
检测到已训练的 BBPE 分词器，正在加载...


###### 1.4 Dataset 与 DataLoader 由于 datasets
    库返回的数据结构是字典（例如 {'en': '...', 'de': '...'}），我们需要调整 Dataset 的读取逻辑。

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


# [New] 适配 HF Datasets 的 PyTorch Dataset
class TranslationDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer_mgr, split='train'):
        self.data = hf_dataset[split]  # 获取对应 split (train/validation/test)
        self.tokenizer_mgr = tokenizer_mgr
        self.src_lang = 'de'
        self.trg_lang = 'en'

        # 预先获取特殊 token 的 ID
        self.sos_id = tokenizer_mgr.tokenizer_en.token_to_id("<sos>")
        self.eos_id = tokenizer_mgr.tokenizer_en.token_to_id("<eos>")
        # 假设两个分词器的特殊 token id 是一样的（通常是的），如果不放心可以分别获取

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 1. 获取原始文本
        # HF Dataset item 格式: {'de': '...', 'en': '...'}
        item = self.data[idx]
        src_text = item[self.src_lang]
        trg_text = item[self.trg_lang]

        # 2. 分词并转 ID (Encoding)
        # 这里的 encode().ids 直接返回数字列表，不需要再查 Vocab 了！
        # BPE 分词器自带 Vocab 功能
        src_ids = self.tokenizer_mgr.tokenizer_de.encode(src_text).ids
        trg_ids = self.tokenizer_mgr.tokenizer_en.encode(trg_text).ids

        # 3. 添加 <sos> 和 <eos>
        src_tensor = torch.tensor([self.sos_id] + src_ids + [self.eos_id], dtype=torch.long)
        trg_tensor = torch.tensor([self.sos_id] + trg_ids + [self.eos_id], dtype=torch.long)

        return src_tensor, trg_tensor


#(保持不变，逻辑通用)
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    # 获取 pad_id (从分词器中获取)
    pad_id = tokenizer_manager.tokenizer_en.token_to_id("<pad>")

    src_padded = pad_sequence(src_batch, padding_value=pad_id, batch_first=True)
    trg_padded = pad_sequence(trg_batch, padding_value=pad_id, batch_first=True)

    return src_padded, trg_padded


class StandardDataLoader:
    def __init__(self, tokenizer_manager):
        self.tm = tokenizer_manager

    def make_loaders(self, batch_size, device):
        print('正在创建 DataLoader...')
        hf_data = self.tm.get_dataset()

        # 创建 Dataset 实例
        train_ds = TranslationDataset(hf_data, self.tm, 'train')
        valid_ds = TranslationDataset(hf_data, self.tm, 'validation')
        test_ds = TranslationDataset(hf_data, self.tm, 'test')

        # 创建 DataLoader
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=0)
        valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=0)
        test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=0)

        # Wrapper 保持接口兼容
        class BatchWrapper:
            def __init__(self, src, trg):
                self.src = src.to(device)
                self.trg = trg.to(device)

        def wrap(loader):
            for src, trg in loader:
                yield BatchWrapper(src, trg)

        return wrap(train_loader), wrap(valid_loader), wrap(test_loader)


# 运行加载
loader_helper = StandardDataLoader(tokenizer_manager)
train_iter, valid_iter, test_iter = loader_helper.make_loaders(batch_size=batch_size, device=device)


正在创建 DataLoader...


In [12]:
# 验证数据
print('\n---- 验证数据加载 ----')
batch = next(train_iter)
print("Src Shape:", batch.src.shape)
print("Trg Shape:", batch.trg.shape)
print("Src Example (IDs):", batch.src[0])

# 获取词表大小供后续模型使用
enc_voc_size = tokenizer_manager.tokenizer_de.get_vocab_size()
dec_voc_size = tokenizer_manager.tokenizer_en.get_vocab_size()
print(f"Encoder Vocab Size: {enc_voc_size}")
print(f"Decoder Vocab Size: {dec_voc_size}")


---- 验证数据加载 ----
Src Shape: torch.Size([128, 32])
Trg Shape: torch.Size([128, 32])
Src Example (IDs): tensor([   1,  271,  400,  912,   15,  286,  277,  969,  354, 4463,  261,  624,
         354,  519,   17,    2,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], device='cuda:0')
Encoder Vocab Size: 10000
Decoder Vocab Size: 10000


##### 2. Transformer 核心组件构建


###### 2.1 词编码与位置编码 (Embedding & PE)
        1.TokenEmbedding: 负责 word -> vector。
        2.PositionalEncoding: 负责注入位置信息 (sin/cos)。
        3.TransformerEmbedding: 把上面两者加起来，并加上 Dropout。

In [15]:
import torch
import torch.nn as nn
import math
# ===================================================================
# 2.1.1 Token Embedding (词嵌入层)
# ===================================================================
class TokenEmbedding(nn.Embedding):
    """
        Standard Token Embedding Layer
        目的：将 1 个 token (int) 转成一串向量 (vector)
    """
    def __init__(self, vocab_size,d_model, padding_idx=None):
        super(TokenEmbedding, self).__init__(vocab_size,d_model,padding_idx=padding_idx)

# ===================================================================
# 2.1.2 Positional Encoding (位置编码)
# ===================================================================
class PositionalEncoding(nn.Module):
    """
    Positional Encoding Layer
    [User Comment] Position 编码公式 ... (1,1,0,1) -> (sin, cos...)
    作用：因为 Transformer 是并行处理的，没有 RNN 那样的时序感，
    所以需要人为加入位置信息。
    """
    def __init__(self, d_model, max_len, device):
        super(PositionalEncoding, self).__init__()

        # 初始化位置编码矩阵 [max_len, d_model]
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False  # [Standard] 位置编码是固定的，不需要训练更新

        # 生成位置索引 [0, 1, 2, ..., max_len-1] -> [max_len, 1]
        pos = torch.arange(0, max_len, device=device).float().unsqueeze(1)

        # 计算分母中的 term: 10000^(2i/d_model)
        # 使用 log 空间计算更数值稳定: exp(2i * -log(10000) / d_model)
        _2i = torch.arange(0, d_model, 2, device=device).float()

        # [User Comment] 按照公式计算
        # PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        # PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))

    def forward(self, x):
        # x shape: [batch_size, seq_len] 或 [batch_size, seq_len, d_model]
        # 我们这里只需要 seq_len
        seq_len = x.size(1)

        # [Standard] 取出对应长度的位置编码
        # return shape: [seq_len, d_model]
        return self.encoding[:seq_len, :]

# ===================================================================
# 2.1.3 Transformer Embedding (整体封装)
# ===================================================================
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model,
                 max_len,
                 drop_prob, device, padding_idx=None):
        super(TransformerEmbedding, self).__init__()

        # 1. 词嵌入
        self.tok_emb = TokenEmbedding(vocab_size, d_model, padding_idx=padding_idx)

        # 2. 位置编码
        self.pos_emb = PositionalEncoding(d_model, max_len, device)

        # 3. Dropout
        self.drop_out = nn.Dropout(p=drop_prob)

        # 4. 缩放因子
        # 论文中提到 embedding 输出需要乘以 sqrt(d_model)，让数值变大一点，
        # 以便和 positional encoding (范围 -1~1) 相加时，词的信息不会被淹没。
        self.scale = math.sqrt(d_model)

    def forward(self, x):
        # x: [batch_size, seq_len]

        # step 1: 词向量 + 缩放
        token_embeddings = self.tok_emb(x) * self.scale

        # step 2: 获取位置编码
        position_embeddings = self.pos_emb(x)

        # step 3: 相加 (广播机制: [batch, len, dim] + [len, dim])
        input_embeddings = token_embeddings + position_embeddings

        # step 4: Dropout
        return self.drop_out(input_embeddings)

In [19]:
#验证编码是否正确
import torch
import numpy as np

def verify_transformer_embedding_noplot():
    print("开始验证 Transformer Embedding 模块...")

    # ==========================
    # 1. 基础参数设置
    # ==========================
    vocab_size = 1000
    d_model = 512
    max_len = 100
    drop_prob = 0.1
    padding_idx = 0

    # 自动选择设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"   设备: {device}")

    try:
        # ==========================
        # 2. 实例化模型
        # ==========================
        # 假设 TransformerEmbedding 类已经定义在上下文中
        model = TransformerEmbedding(
            vocab_size=vocab_size,
            d_model=d_model,
            max_len=max_len,
            drop_prob=drop_prob,
            device=device,
            padding_idx=padding_idx
        ).to(device)
        print("    模型实例化成功")

        # ==========================
        # 3. 维度形状检查 (Shape Check)
        # ==========================
        batch_size = 2
        seq_len = 50
        # 构造输入，包含一些 0 (padding)
        input_data = torch.randint(1, vocab_size, (batch_size, seq_len)).to(device)
        input_data[:, -5:] = padding_idx # 最后5个设为 pad

        # 前向传播
        output = model(input_data)

        expected_shape = (batch_size, seq_len, d_model)
        if output.shape == expected_shape:
            print(f"    输出维度检查通过: {output.shape}")
        else:
            print(f"    输出维度错误: 期望 {expected_shape}, 实际 {output.shape}")
            return

        # ==========================
        # 4. Padding 逻辑检查
        # ==========================
        # 检查 TokenEmbedding 层的权重，padding_idx 对应的行应该是全 0
        emb_weight = model.tok_emb.weight.data.cpu()
        if torch.equal(emb_weight[padding_idx], torch.zeros(d_model)):
            print("    Padding Index 逻辑通过: Token Embedding 中 Pad 对应的向量全为 0")
        else:
            print("    Padding Index 逻辑失败: Pad 向量不为 0，请检查 padding_idx 是否传入 super().__init__")

        print("🎉 验证全部完成！Embedding 模块逻辑正常。")

    except Exception as e:
        print(f"\n 验证过程中发生错误: {e}")
        import traceback
        traceback.print_exc()

# 运行验证
verify_transformer_embedding_noplot()

开始验证 Transformer Embedding 模块...
   设备: cuda
    模型实例化成功
    输出维度检查通过: torch.Size([2, 50, 512])
    Padding Index 逻辑通过: Token Embedding 中 Pad 对应的向量全为 0
🎉 验证全部完成！Embedding 模块逻辑正常。


######  2.2 Mask 掩码机制 (至关重要)


In [20]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    """
    多头注意力的目的是让模型可以从不同的子空间（different subspaces）学习信息。
    比如：一个头关注语法关系，另一个头关注语义指代。
    """
    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.d_head = d_model // n_head # 每个头的维度 (512 / 8 = 64)

        # [Standard] 强制检查 d_model 是否能被 n_head 整除
        assert d_model % n_head == 0, f"d_model ({d_model}) must be divisible by n_head ({n_head})"

        # 定义 W_q, W_k, W_v 矩阵 (使用全连接层实现)
        # 这里的输入输出都是 d_model，具体的“分头”逻辑在 forward 里做
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        # 定义最后输出的线性层 W_o
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        """
        q, k, v: [batch_size, seq_len, d_model]
        mask:    [batch_size, 1, 1, seq_len] (或类似广播维度)
        """
        batch_size = q.size(0)

        # =================================================
        # 1. 线性变换 + 分头 (Split Heads)
        # =================================================
        # 原始: [batch, len, d_model]
        # 变换: [batch, len, n_head, d_head]
        # 转置: [batch, n_head, len, d_head] -> 这一步是为了让 n_head 维度在前面，方便并行计算
        Q = self.w_q(q).view(batch_size, -1, self.n_head, self.d_head).transpose(1, 2)
        K = self.w_k(k).view(batch_size, -1, self.n_head, self.d_head).transpose(1, 2)
        V = self.w_v(v).view(batch_size, -1, self.n_head, self.d_head).transpose(1, 2)

        # =================================================
        # 2. 缩放点积注意力 (Scaled Dot-Product Attention)
        # =================================================
        # 2.1 计算注意力分数 scores = Q * K^T / sqrt(d_k)
        # MatMul: [batch, head, len_q, d] x [batch, head, d, len_k] -> [batch, head, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_head)

        # 2.2 应用 Mask (核心逻辑)
        if mask is not None:
            # [User Comment] mask 为 0 的位置替换为极小值 (-1e9)
            # 这样 Softmax 后这些位置的概率趋近于 0，模型就不会关注它们
            scores = scores.masked_fill(mask == 0, -1e9)

        # 2.3 Softmax 归一化 (得到注意力权重)
        attn_weights = torch.softmax(scores, dim=-1)

        # 2.4 加权求和: weights * V
        # [batch, head, len_q, len_k] x [batch, head, len_k, d_head] -> [batch, head, len_q, d_head]
        context = torch.matmul(attn_weights, V)

        # =================================================
        # 3. 拼接多头 (Concat Heads)
        # =================================================
        # [batch, head, len, d_head] -> [batch, len, head, d_head] -> [batch, len, d_model]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # =================================================
        # 4. 最终线性输出 (Final Linear)
        # =================================================
        output = self.fc_out(context)

        return output, attn_weights

In [21]:
# --- 验证 2.2 Attention 模块 ---
print("\n-------- 测试 2.2 Multi-Head Attention --------")

# 1. 模拟参数
d_model = 512
n_head = 8
batch_size = 2
seq_len = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 实例化
mha = MultiHeadAttention(d_model=d_model, n_head=n_head).to(device)

# 3. 模拟输入 (假设 Q, K, V 都是同一个输入，即 Self-Attention)
x = torch.randn(batch_size, seq_len, d_model).to(device)

# 4. 模拟 Mask (batch=2, 1, 1, len=10)
# 假设最后两个词是 padding (0)
mask = torch.ones(batch_size, 1, 1, seq_len).to(device)
mask[:, :, :, -2:] = 0 # 设为遮挡

# 5. 前向传播
output, weights = mha(x, x, x, mask=mask)

print(f"输入形状: {x.shape}")        # [2, 10, 512]
print(f"输出形状: {output.shape}")   # [2, 10, 512] (应与输入一致)
print(f"权重形状: {weights.shape}")  # [2, 8, 10, 10] (Batch, Head, Q_Len, K_Len)

if output.shape == (batch_size, seq_len, d_model):
    print(" Attention 模块维度验证通过！")


-------- 测试 2.2 Multi-Head Attention --------
输入形状: torch.Size([2, 10, 512])
输出形状: torch.Size([2, 10, 512])
权重形状: torch.Size([2, 8, 10, 10])
 Attention 模块维度验证通过！


###### 2.3 前馈神经网络 (Feed-Forward)：给模型增加非线性变换能力。

In [ ]:
# ===================================================================
# 2.3 Position-wise Feed-Forward Network (前馈神经网络)
# ===================================================================
class PositionwiseFeedForward(nn.Module):
    """
    这是一个两层的全连接网络，中间包含 ReLU 激活函数。
    它对序列中的每一个位置 (position) 独立且相同地进行处理。
    公式: FFN(x) = max(0, xW1 + b1)W2 + b2
    """
    def __init__(self, d_model, ffn_hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # d_model: 输入/输出维度 (512)
        # ffn_hidden: 隐藏层维度 (2048) - 通常是 d_model 的 4 倍
        self.net = nn.Sequential(
            nn.Linear(d_model, ffn_hidden),
            nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Linear(ffn_hidden, d_model)
        )

    def forward(self, x):
        # x shape: [batch_size, seq_len, d_model]
        return self.net(x)